In [ ]:
#libraries to use for this exercise (please fun this cell first):
import numpy as np
from matplotlib import pyplot as plt
import scipy.optimize as opt
import scipy.io
import matplotlib.cm as cm

In [ ]:
#Load the .mat data using scipy.io, which will be a dictionary datatype.
data_dict = scipy.io.loadmat('ex4/ex4data1.mat')
X=data_dict['X']
y=data_dict['y']
X=np.matrix(X)
y=np.matrix(y)

In [ ]:
#plt.imsave('filename.png', np.array(X[2000:2100,:]).reshape(200,200), cmap=cm.gray)
print (plt.imshow(np.array(X[130,:]).reshape(20,20),cmap=cm.gray))

In [ ]:
#Load the weights
weights = scipy.io.loadmat('ex4/ex4weights.mat')
Theta1=weights['Theta1']
Theta2=weights['Theta2']
print (type(Theta1))

In [ ]:
#define a cost function to calculate cost of the neural networks
def sigmoid(z):
    g = np.zeros(np.shape(z))
    g = 1/(1+np.exp(-z))
    return g
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambd):
    #extract Theta1 and Theta2 from nn_params and note nn_params is an array type, we need Thetas to be matrix
    Theta1 = nn_params[0:hidden_layer_size*(input_layer_size+1)]
    Theta1 = Theta1.reshape(hidden_layer_size, input_layer_size+1)
    Theta1 = np.matrix(Theta1)
    Theta2 = nn_params[hidden_layer_size*(input_layer_size+1):]
    Theta2 = Theta2.reshape(num_labels, hidden_layer_size + 1)
    Theta2 = np.matrix(Theta2)
    #initiaalize gradient for Thetas and they are arrays
    Theta1_grad = np.zeros((np.shape(Theta1)))
    Theta2_grad = np.zeros((np.shape(Theta2)))
    #set up some useful variables
    #Note X and y should be matrix after adding the bias still
    [m, n] = np.shape(X)
    additional_coulmn = np.ones((m,1))
    X = np.append(additional_coulmn,X,axis=1)
    #forward propagation calculation starting from input layer to hidden layer
    Layer2 = X*Theta1.T
    Layer2 = sigmoid(Layer2)
    additional_coulmn = np.ones((m,1))
    Layer2 = np.append(additional_coulmn,Layer2,axis=1)
    #forward propagation calculation starting from hidden layer to output layer
    Layer3 = Layer2*Theta2.T
    predict = sigmoid(Layer3)
    #transform y to be a matrix where each row's index corresponds to the correct label
    ychange = np.zeros((m, num_labels))
    for i in range(0,m):
        row = ychange[i,:]
        row[int(y[i])-1]=1
        ychange[i,:]=row
    ychange = np.matrix(ychange)
    #calculate the cost    
    J=(np.sum(np.multiply((-ychange),np.log(predict))-np.multiply((1-ychange),np.log(1-predict))))/m+(np.sum(np.multiply((Theta1[:,1:input_layer_size+1]),((Theta1[:,1:input_layer_size+1]))))+np.sum(np.multiply((Theta2[:,1:hidden_layer_size+1]),(Theta2[:,1:hidden_layer_size+1]))))*lambd/(2*m)

    #(from matlab)J=(sum((-(log(predict)).*ychange-(log(1-predict)).*(-ychange+1)),'all'))/m+(sum(Theta1(:,[2:(size(Theta1,2))]).*Theta1(:,[2:(size(Theta1,2))]),'all')+sum(Theta2(:,[2:(size(Theta2,2))]).*Theta2(:,[2:(size(Theta2,2))]),'all'))*lambda/(2*m);
    #(from logistic)J = np.sum(np.dot((-y.T),np.log(h))-np.dot((1-y).T,np.log(1-h)))/m + np.matrix(theta[1:])*np.transpose(np.matrix(theta[1:]))*lambd/(2*m)
    
    #back propagation: for each row (each prediction), calculate the grad for Theta1 and Theta2
    for i in range (0,m):
        #take out the first row of the input data
        a1=X[i,:]
        #calculate the hiddden layer
        z2=a1*Theta1.T
        a2=sigmoid(z2)
        #add bias for calculating output layer
        additional_coulmnfora2 = np.ones((np.shape(a2)[0],1))
        a2 = np.append(additional_coulmnfora2,a2,axis=1)
        z3=a2*Theta2.T
        a3=sigmoid(z3)
        #create a matrix where the row value == 1 indicates the prediction
        yvec = np.zeros((1,num_labels))
        yvec[0][int(y[i])-1]=1
        yvec = np.matrix(yvec)
        #back propagation
        error3=a3-yvec
        error2=np.multiply(Theta2.T*error3.T,(np.multiply(a2,(1-a2))).T)
        Theta2_grad = Theta2_grad+error3.T*a2
        Theta1_grad = Theta1_grad+error2[1:]*a1
    #remove bias term's grad
    Theta2passon = Theta2[:,1:hidden_layer_size+1]
    additional_coulmnforTheta2passon = np.zeros((np.shape(Theta2passon)[0],1))
    Theta2passon = np.append(additional_coulmnforTheta2passon,Theta2passon,axis=1)
    Theta2_grad = Theta2_grad/m+(lambd/m)*Theta2passon
    
    Theta1passon = Theta1[:,1:input_layer_size+1]
    additional_coulmnforTheta1passon = np.zeros((np.shape(Theta1passon)[0],1))
    Theta1passon = np.append(additional_coulmnforTheta1passon,Theta1passon,axis=1)
    Theta1_grad = Theta1_grad/m+(lambd/m)*Theta1passon
    
    grad = np.append(np.array(Theta1_grad), np.array(Theta2_grad))
    
    return J, grad

In [ ]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10

nn_params = np.append(np.array(Theta1), np.array(Theta2))

lambd = 1

result = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambd)

print (result)

In [ ]:
#randomly initialise weights
def randInitializeWeights(L_in, L_out):
    epsilon_init = 0.12;
    W = np.random.rand(L_out, L_in+1) * 2 * epsilon_init - epsilon_init
    return W

In [ ]:
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);

In [ ]:
#find the optimal theta
initial_nn_params = np.append(np.array(initial_Theta1), np.array(initial_Theta2))
lambd =1
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
result = opt.fmin_tnc(func=nnCostFunction, x0=np.array(initial_nn_params), args=(input_layer_size, hidden_layer_size, num_labels, X, y, lambd))
print('Thetas found by fmin_tnc function: ', result)
print('theta: \n',result[0])

In [ ]:
print(np.shape(result[0]))

In [ ]:
nn=np.array(result[0])

In [ ]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10

lambd = 1

result2 = nnCostFunction(nn, input_layer_size, hidden_layer_size, num_labels, X, y, lambd)

print (result2)